# Week 3 Assignment 


Sayaka Minegishi

## Part 1

#### Import packages

In [2]:
!pip install bs4

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes


In [3]:
!pip install pandas==1.0.3

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes


#### Load libraries

In [4]:
from bs4 import BeautifulSoup
import requests #used to download a webpage
import pandas as pd
import numpy as np

Conduct webscraping from https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

In [5]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M' #retrieve url
torontodata = requests.get(url).text #get the webpage contents in text format
soup = BeautifulSoup(torontodata, "html5lib") #create a beautiful soup object

#create a new list
table_contents = []

#find table and table data
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass #ignore cells with a borough that is not assigned
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        
        if cell['Neighborhood'] == 'Not assigned':
            cell['Neighborhood'] =  cell['Borough']
    
        
        
        table_contents.append(cell)
        



#create a dataframe with the list

toronto_df=pd.DataFrame(table_contents)



toronto_df['Borough']=toronto_df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})




The dataframe:

In [6]:
toronto_df

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto Business,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [7]:
#print the number of rows of the dataframe
print(toronto_df.shape)
print("The dataframe has", toronto_df.shape[0], "rows")

(103, 3)
The dataframe has 103 rows


## Part 2

Using the location data to create a new dataframe with the latitude and longitude values for each neighborhood.

In [8]:
geourl = 'https://cocl.us/Geospatial_data' #url for the csv file containing location data

location_df = pd.read_csv(geourl) #store csv data to a new dataframe

location_df.rename(columns = {'Postal Code':'PostalCode'}, inplace = True) #rename the postal code column for location_df





In [9]:
#merge the two dataframes
toronto_df_wlocation = pd.merge(toronto_df, location_df, on='PostalCode') 
toronto_df_wlocation #display the new merged dataframe

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto Business,Enclave of M4L,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


## Part 3

Import packages:

In [10]:
pip install folium

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
Note: you may need to restart the kernel to use updated packages.


In [11]:
import json
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize #transform JSON file to a pandas df

import matplotlib.cm as cm
import matplotlib.colors as colors

import folium #for map rendering
print('libraries imported')


libraries imported


#### Analysis

First get the geographical coordinates of Toronto and store the latitude and longitude vales.

In [12]:
#get the geographical coordinates of Toronto
address = 'Toronto, Canada'
geolocator = Nominatim(user_agent = "toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinates of Toronto are {}, {}.'.format(latitude, longitude))


The geographical coordinates of Toronto are 43.6534817, -79.3839347.


Create a map of the neighborhoods of Toronto, where each neighborhood is represented as a point on the map.

In [13]:
#create a map of Toronto and its neighborhoods

toronto_map = folium.Map(location = [latitude, longitude], zoom_start= 10)

#add markers
for lat, lng, borough, neighborhood in zip(toronto_df_wlocation['Latitude'],toronto_df_wlocation['Longitude'], toronto_df_wlocation['Borough'],toronto_df_wlocation['Neighborhood']):
    #define the labels and their properties
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html= True)
    
    #add a point
    folium.CircleMarker(
        [lat, lng],
        radius = 5,
        popup = label,
        color = 'blue',
        fill = True,
        fill_color = '#3186cc',
        fill_opacity = 0.7,
        parse_html = False).add_to(toronto_map)

toronto_map #display the map of Toronto with all the neighborhoods marked






I will now look only at neighborhoods with a borough that contains the word 'Toronto'. I will first create a new dataframe that stores this data, called toronto_only_df.

In [14]:
#create a dataframe that only contains data for boroughs that contain the word "Toronto"

toronto_only_df = toronto_df_wlocation[toronto_df_wlocation['Borough'].str.contains('Toronto')]

I visualize the neighborhoods with a borough that contains the word 'Toronto' on a map. Each point on the map represents a neighborhood.

In [15]:
toronto_map_focused = folium.Map(location = [latitude, longitude], zoom_start= 10)

#add markers
for lat, lng, borough, neighborhood in zip(toronto_only_df['Latitude'],toronto_only_df['Longitude'], toronto_only_df['Borough'],toronto_only_df['Neighborhood']):
    #define the labels and their properties
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html= True)
    
    #add a point
    folium.CircleMarker(
        [lat, lng],
        radius = 5,
        popup = label,
        color = 'blue',
        fill = True,
        fill_color = '#3186cc',
        fill_opacity = 0.7,
        parse_html = False).add_to(toronto_map_focused)

toronto_map_focused #display the map of Toronto with all the neighborhoods marked



I define my Foursquare credentials and version:

In [16]:
CLIENT_ID = 'HU4NW3X5GFHOKALM3VMUHXFC5GBEE04KCAEYUEYZOTHQYU0U' #  Foursquare ID
CLIENT_SECRET = 'EVWCF1AOGUWLPFPKWYUOQJZ34BPCTMNNRRVOB3I4CXOZJ00N' #  Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # default Foursquare API limit value

print('Foursquare credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Foursquare credentails:
CLIENT_ID: HU4NW3X5GFHOKALM3VMUHXFC5GBEE04KCAEYUEYZOTHQYU0U
CLIENT_SECRET:EVWCF1AOGUWLPFPKWYUOQJZ34BPCTMNNRRVOB3I4CXOZJ00N


Now a function will be created to explore all the neighborhoods in Toronto that will give us the top 100 venues that are in each neighborhood within 500 meters radius. We store these venues in a new dataframe called nearby_venues.

In [17]:
def get_nearby_venues(neighborhoodnames, latitudes, longitudes, radius = 500):
    venues_list = [] #create an empty list to store the venues
    
    for name, lat, lng in zip(neighborhoodnames, latitudes, longitudes): #zip creates a list of tuples in the form (neighborhood name, latitude, longitude)
        print(name)
        
        #create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)
        
        #make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        #return only the relevant information for each venue
        venues_list.append([(
            name,
            lat,
            lng,
            v['venue']['name'],
            v['venue']['location']['lat'],
            v['venue']['location']['lng'],
            v['venue']['categories'][0]['name']) for v in results])
        
        
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        
    nearby_venues.columns = ['Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude', 'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category']
        
    return(nearby_venues)
                                           
        
        
        

Get nearby venues for each neighborhood in Toronto using the function defined in the above cell, and store the data in a new dataframe called venues_toronto_neighborhoods.

In [18]:
#create a new dataframe, venues_toronto_neighborhoods

venues_toronto_neighborhoods = get_nearby_venues(neighborhoodnames = toronto_only_df['Neighborhood'], latitudes =toronto_only_df['Latitude'], longitudes = toronto_only_df['Longitude']) 
                                              

Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
The Danforth  East
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Enclave of M5E
St. James Town, Cabbagetown
First Canadi

The number of rows and columns in the dataframe venues_toronto_neighborhoods:

In [19]:
print(venues_toronto_neighborhoods.shape)

(1580, 7)


Check the first 5 rows of the dataframe, venues_toronto_neighborhoods:

In [20]:
venues_toronto_neighborhoods.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
1,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant
4,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


Display the number of venues for each neighborhood:

In [21]:
venues_toronto_neighborhoods.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,58,58,58,58,58,58
"Brockton, Parkdale Village, Exhibition Place",22,22,22,22,22,22
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",16,16,16,16,16,16
Central Bay Street,62,62,62,62,62,62
Christie,16,16,16,16,16,16
Church and Wellesley,72,72,72,72,72,72
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,36,36,36,36,36,36
Davisville North,9,9,9,9,9,9


Find the number of unique venue categories that were found in the results:

In [22]:
print("The number of unique venue categories is {}.".format(len(venues_toronto_neighborhoods['Venue Category'].unique())))

The number of unique venue categories is 231.


Conduct analysis of each of the neighborhoods in Toronto. First we apply one hot encoding on the venue categories.

In [23]:
#use one hot encoding on Toronto venue categories
toronto_onehot = pd.get_dummies(venues_toronto_neighborhoods[['Venue Category']], prefix = "", prefix_sep = "")

#add the neighborhood column to toronto_onehot
toronto_onehot['Neighborhood'] = venues_toronto_neighborhoods['Neighborhood']

#move the neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head() #display the first 5 rows of the new dataframe



,Yoga Studio,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,...,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


We group each of the rows by neighborhood and by the mean of the frequency of occurrence of each category.

In [24]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()

toronto_grouped #display

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,Berczy Park,0.000000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,0.00,...,0.000000,0.00000,0.000000,0.00000,0.00,0.017241,0.000000,0.000000,0.000000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,0.00,...,0.000000,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
2,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.0625,0.0625,0.125,0.125,0.125,0.000000,0.000000,0.00,...,0.000000,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
3,Central Bay Street,0.016129,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,0.00,...,0.000000,0.00000,0.000000,0.00000,0.00,0.016129,0.000000,0.000000,0.016129,0.000000
4,Christie,0.000000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,0.00,...,0.000000,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
5,Church and Wellesley,0.027778,0.0000,0.0000,0.000,0.000,0.000,0.013889,0.000000,0.00,...,0.013889,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.013889
6,"Commerce Court, Victoria Hotel",0.000000,0.0000,0.0000,0.000,0.000,0.000,0.030000,0.000000,0.00,...,0.000000,0.00000,0.000000,0.00000,0.00,0.020000,0.000000,0.000000,0.010000,0.000000
7,Davisville,0.000000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,0.00,...,0.000000,0.00000,0.027778,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
8,Davisville North,0.000000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,0.00,...,0.000000,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
9,"Dufferin, Dovercourt Village",0.000000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,0.00,...,0.000000,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000


Create a pandas dataframe with the 10 most common venues for each neighborhood:

In [25]:
#define function that returns the most common venues

def return_most_common_venues(row, num_of_venues):
    row_categories = row.iloc[1: ]
    row_categories_sorted = row_categories.sort_values(ascending = False) #sort the values in descending order
    return row_categories_sorted.index.values[0: num_of_venues]

    

Make a new dataframe to display the top 10 venues for each neighborhood:

In [26]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

#create columns according to the number of top venues
columns = ['Neighborhood']

for i in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(i + 1, indicators[i]))
    except:
        columns.append('{}th Most Common Venue'.format(i+1))
    
#creaet a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns = columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for j in np.arange(toronto_grouped.shape[0]): #ie. for each row in toronto_grouped
    neighborhoods_venues_sorted.iloc[j, 1: ]= return_most_common_venues(toronto_grouped.iloc[j, :], num_top_venues)
    
neighborhoods_venues_sorted.head() #display the first 5 neighborhoods
        

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Bakery,Cocktail Bar,Restaurant,Beer Bar,Seafood Restaurant,Pharmacy,Farmers Market,Cheese Shop,Park
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Breakfast Spot,Stadium,Intersection,Bar,Bakery,Restaurant,Climbing Gym,Furniture / Home Store
2,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport Terminal,Boutique,Boat or Ferry,Rental Car Location,Coffee Shop,Plane,Bar,Harbor / Marina
3,Central Bay Street,Coffee Shop,Sandwich Place,Italian Restaurant,Café,Burger Joint,Thai Restaurant,Salad Place,Bubble Tea Shop,Park,Poke Place
4,Christie,Grocery Store,Café,Park,Nightclub,Candy Store,Baby Store,Italian Restaurant,Athletics & Sports,Restaurant,Coffee Shop


We will now cluster the neighborhoods by running k-means that will separate the neighborhood into 5 clusters.

In [27]:
from sklearn.cluster import KMeans #import k means

#number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood',1) #drop the Neighborhood column

#make a new model for k means clustering 
kmeans = KMeans(n_clusters = kclusters, random_state = 4).fit(toronto_grouped_clustering)

#see the cluster labels for each row in the dataframe
kmeans.labels_[0:10]


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

Make a new dataframe that has the cluster label and the top 10 venues for each neighborhood.

In [28]:
#add cluster labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_only_df

#merge toronto_grouped dataframe with toronto_only_df 

toronto_merged= toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() #display the first 5 rows
                                    
                                    
                                    

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Pub,Bakery,Park,Breakfast Spot,Café,Theater,Gym / Fitness Center,Farmers Market,Restaurant
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Clothing Store,Coffee Shop,Cosmetics Shop,Bubble Tea Shop,Middle Eastern Restaurant,Café,Diner,Japanese Restaurant,Italian Restaurant,Ramen Restaurant
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Café,Cocktail Bar,Cosmetics Shop,Beer Bar,Seafood Restaurant,Farmers Market,Restaurant,Italian Restaurant,Gym
19,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Pub,Asian Restaurant,Health Food Store,Trail,Wings Joint,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Electronics Store
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Coffee Shop,Bakery,Cocktail Bar,Restaurant,Beer Bar,Seafood Restaurant,Pharmacy,Farmers Market,Cheese Shop,Park


Visualize the clusters on a map:

In [29]:
map_toronto_clusters = folium.Map(location=[latitude, longitude], zoom_start=11) # make a new map

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.6).add_to(map_toronto_clusters)
       
map_toronto_clusters #display map

we now examine each cluster:

CLUSTER 1:

In [30]:
toronto_merged.loc[toronto_merged['Cluster Labels']==0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
                                                                                            

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,0,Coffee Shop,Pub,Bakery,Park,Breakfast Spot,Café,Theater,Gym / Fitness Center,Farmers Market,Restaurant
9,Downtown Toronto,0,Clothing Store,Coffee Shop,Cosmetics Shop,Bubble Tea Shop,Middle Eastern Restaurant,Café,Diner,Japanese Restaurant,Italian Restaurant,Ramen Restaurant
15,Downtown Toronto,0,Coffee Shop,Café,Cocktail Bar,Cosmetics Shop,Beer Bar,Seafood Restaurant,Farmers Market,Restaurant,Italian Restaurant,Gym
19,East Toronto,0,Pub,Asian Restaurant,Health Food Store,Trail,Wings Joint,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Electronics Store
20,Downtown Toronto,0,Coffee Shop,Bakery,Cocktail Bar,Restaurant,Beer Bar,Seafood Restaurant,Pharmacy,Farmers Market,Cheese Shop,Park
24,Downtown Toronto,0,Coffee Shop,Sandwich Place,Italian Restaurant,Café,Burger Joint,Thai Restaurant,Salad Place,Bubble Tea Shop,Park,Poke Place
25,Downtown Toronto,0,Grocery Store,Café,Park,Nightclub,Candy Store,Baby Store,Italian Restaurant,Athletics & Sports,Restaurant,Coffee Shop
30,Downtown Toronto,0,Coffee Shop,Café,Restaurant,Deli / Bodega,Gym,Thai Restaurant,Clothing Store,Salad Place,Cosmetics Shop,Hotel
31,West Toronto,0,Pharmacy,Bakery,Middle Eastern Restaurant,Music Venue,Park,Pet Store,Café,Brewery,Bar,Bank
36,Downtown Toronto,0,Coffee Shop,Aquarium,Café,Hotel,Restaurant,Sporting Goods Shop,Scenic Lookout,Italian Restaurant,Fried Chicken Joint,Brewery


CLUSTER 2:

In [32]:
toronto_merged.loc[toronto_merged['Cluster Labels']==1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
                                                                                            

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
35,East York/East Toronto,1,Park,Convenience Store,Intersection,Wings Joint,Distribution Center,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store
91,Downtown Toronto,1,Park,Playground,Trail,Wings Joint,Diner,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant


CLUSTER 3:

In [33]:
toronto_merged.loc[toronto_merged['Cluster Labels']==2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
                                                                                            

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
62,Central Toronto,2,Garden,Home Service,Ice Cream Shop,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant


CLUSTER 4:

In [34]:
toronto_merged.loc[toronto_merged['Cluster Labels']==3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
                                                                                            

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
61,Central Toronto,3,Park,Bus Line,Swim School,Wings Joint,Distribution Center,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store
68,Central Toronto,3,Park,Trail,Jewelry Store,Bus Line,Sushi Restaurant,Wings Joint,Distribution Center,Event Space,Ethiopian Restaurant,Escape Room


CLUSTER 5:

In [37]:
toronto_merged.loc[toronto_merged['Cluster Labels']==4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
                                                                                            

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
83,Central Toronto,4,Park,Tennis Court,Restaurant,Wings Joint,Diner,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant
